In [2]:
import earthaccess
import xarray as xr
import pandas as pd
import numpy as np
from tqdm import tqdm

auth = earthaccess.login()
bounds = (-120.75, 33.5, -119, 34.5) 

df_kelp = pd.read_csv("../../data/PISCO_kelpdata.csv")
df_kelp['time'] = pd.to_datetime(df_kelp[['year', 'month', 'day']])
df_kelp['sst'] = np.nan

sst_query = earthaccess.search_data(
    short_name="MUR-JPL-L4-GLOB-v4.1",
    bounding_box=bounds,
    temporal=("2024-04-01", "2024-12-31")
)

urls = earthaccess.open(sst_query)


for url in tqdm(urls):
    try:
        with xr.open_dataset(url, engine="h5netcdf") as ds:
            # Match formats (removing timezone for safety)
            current_day_dt = pd.to_datetime(ds.time.values[0]).replace(tzinfo=None)
            
            # Mask the kelp dataframe for rows matching this day
            day_mask = (df_kelp['time'].dt.normalize() == current_day_dt.normalize())
            rows_today = df_kelp[day_mask]
            
            if not rows_today.empty:
                day_sst = ds.analysed_sst.sel(
                    lat=slice(bounds[1], bounds[3]), 
                    lon=slice(bounds[0], bounds[2])
                ) - 273.15
                
                # We iterate only through the rows that match today's date
                for idx, row in rows_today.iterrows():
                    val = day_sst.sel(lat=row.latitude, lon=row.longitude, method='nearest').values
                    # Update the existing dataframe directly
                    df_kelp.at[idx, 'sst'] = float(val)
                    
    except Exception as e:
        print(f"Error on file {url}: {e}")


df_kelp.to_csv("temp_sst_lookup.csv", index=False)
print(f"vinal count of matched SST values: {df_kelp['sst'].notna().sum()}")

QUEUEING TASKS | :   0%|          | 0/276 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/276 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/276 [00:00<?, ?it/s]

  0%|          | 0/276 [00:00<?, ?it/s]/tmp/ipykernel_1886/3744987422.py:25: FutureWarning: In a future version, xarray will not decode the variable 'dt_1km_data' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence of a timedelta64 'dtype' attribute, which is now xarray's default way of encoding timedelta64 values.
To continue decoding into a timedelta64 dtype, either set `decode_timedelta=True` when opening this dataset, or add the attribute `dtype='timedelta64[ns]'` to this variable on disk.
To opt-in to future behavior, set `decode_timedelta=False`.
  with xr.open_dataset(url, engine="h5netcdf") as ds:
  0%|          | 1/276 [00:00<01:55,  2.38it/s]/tmp/ipykernel_1886/3744987422.py:25: FutureWarning: In a future version, xarray will not decode the variable 'dt_1km_data' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence 

vinal count of matched SST values: 4502


In [5]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

# 1. Load data
df_final = pd.read_csv("temp_sst_lookup.csv")
df_final['time'] = pd.to_datetime(df_final['time']).dt.tz_localize(None)

# --- DEBUG BLOCK ---
print(f"Initial rows: {len(df_final)}")
print(f"Rows with SST data: {df_final['sst'].notna().sum()}")

# Identify the correct kelp count column
if 'observation_count' in df_final.columns:
    kelp_col = 'observation_count'
elif 'count' in df_final.columns:
    kelp_col = 'count'
else:
    # If it's something else, find the first column with 'count' in the name
    kelp_col = [c for c in df_final.columns if 'count' in c.lower()][0]
print(f"Using kelp column: {kelp_col}")
# -------------------

# 2. Load CUTI
ds_cuti = xr.open_dataset("../../data/CUTI_daily.nc")
if ds_cuti.indexes['time'].tzinfo is not None:
    ds_cuti['time'] = ds_cuti.indexes['time'].tz_localize(None)

def get_cuti(row):
    try:
        val = ds_cuti.cuti.sel(lat=row.latitude, time=row.time, method='nearest').values
        return float(val)
    except:
        return np.nan

df_final['cuti'] = df_final.apply(get_cuti, axis=1)

# 3. Clean data
# We drop rows only if they are missing the ESSENTIALS
df_plot = df_final.dropna(subset=['sst', 'cuti', kelp_col]).sort_values('time')

print(f"Rows remaining for correlation: {len(df_plot)}")

if len(df_plot) < 2:
    print("!!! ERROR: Not enough data points. Check if your SST extraction actually worked.")
else:
    # 4. Correlation
    r_sst, p_sst = pearsonr(df_plot['sst'], df_plot[kelp_col])
    r_cuti, p_cuti = pearsonr(df_plot['cuti'], df_plot[kelp_col])

    # 5. Plotting
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    sns.regplot(data=df_plot, x='sst', y=kelp_col, ax=ax1, color='red', scatter_kws={'alpha':0.4})
    ax1.set_title(f"SST vs Kelp (r={r_sst:.2f})")

    sns.regplot(data=df_plot, x='cuti', y=kelp_col, ax=ax2, color='green', scatter_kws={'alpha':0.4})
    ax2.set_title(f"CUTI vs Kelp (r={r_cuti:.2f})")

    plt.tight_layout()
    plt.savefig("kelp_health_correlation.png")
    plt.show()
    
    df_plot.to_csv("kelp_health_correlation_final.csv", index=False)

Initial rows: 4502
Rows with SST data: 4502
Using kelp column: count
Rows remaining for correlation: 0
!!! ERROR: Not enough data points. Check if your SST extraction actually worked.
